In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

import os
import pickle

In [2]:
import re
import spacy

### Stopwords

In [3]:
ordinals = ['1st', '2nd', '3rd', '4th']
for i in range(5,31):
    ordinals.append(str(i) + 'th')
    ordinals.append('0' + str(i) + 'th') 
    

In [4]:
number_stopwords = [str(i) for i in range(10001)] + ['0'+str(i) for i in range(100)] + ['000'] + ordinals

In [104]:
df = pd.read_excel('Data/20181001-newindianexpress_sentence_classification_adjudicated_20181218.xlsx')

In [105]:
df = df[np.logical_not(np.isnan(np.array(df['label'])))]

In [106]:
len(df)

8337

In [103]:
for i, row in df.iterrows():
    row['sentence'] = re.sub("http\S*\s", "", row['sentence'])
    row['sentence'] = re.sub("((url)*\s*:\s*)*http\S*", "", row['sentence'])
    
    if row['sentence'] == "":
        df=df.drop([i])
        

In [49]:
len(df)

7796

In [9]:
df1_sents= df['sentence']
df1_y= df['label']

In [10]:
len(df1_sents)

7796

import pickle

data_path = 'Data/first_data.pickle'



prepared_first = {}

prepared_first['X'] = np.array(df1_sents)
prepared_first['y'] = np.array(df1_y)
prepared_first['stopwords'] = np.array(number_stopwords)

with open(data_path, 'wb') as file_:
    pickle.dump(prepared_first, file_, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
## Partition the data

X_, X_test, y_, y_test = train_test_split(np.array(df1_sents), np.array(df1_y), test_size=0.20, random_state=42, stratify=np.array(df1_y))

len(X_), len(X_test)

 

(6236, 1560)

In [12]:
X_tra, X_val, y_tra, y_val = train_test_split(X_, y_, test_size=0.20, random_state=42, stratify=y_)

type(X_tra), len(X_val)

data_path = 'Data/first_data_prepared.pickle'



In [13]:
Data_ = {}

Data_['X_tra'] = X_tra
Data_['y_tra'] = y_tra
Data_['X_val'] = X_val
Data_['y_val'] = y_val
Data_['X_test'] = X_test
Data_['y_test'] = y_test
Data_['stopwords'] = np.array(number_stopwords)

with open(data_path, 'wb') as file_:
    pickle.dump(Data_, file_, protocol=pickle.HIGHEST_PROTOCOL)

## Second Haf of the data

In [14]:
df2 = pd.read_excel('Data/20181126_NewIndianExpress_3Classifier0_Baglan-Ezgi-Balacan-Eylem-Merged_AgreedByAll.xlsx')
df2 = df2[np.logical_not(np.isnan(np.array(df2['label'])))]
df2 = df2[(np.array(df2['label'] == 0))]

In [15]:
nlp = spacy.load('en_core_web_lg')

In [16]:
df2_sents = []
df2_labels = []
for i, row in df2.iterrows():
    row['text'] = row['text'].replace('\n', ' ')
    row['text'] = row['text'].replace('&amp;', 'and')
    row['text'] = row['text'].replace('\xa0', ' ')
    row['text'] = row['text'].replace('-', '')
    row['text'] = row['text'].replace("\'", "")
    row['text'] = row['text'].replace('\"', '')
    row['text'] = row['text'].replace('\“', '')
    row['text'] = re.sub("((url)*\s*:\s*)*http\S*", "", row['text'])
    row['text'] = row['text'].replace('  ', ' ')
    doc = nlp(row['text'])
    for sent in doc.sents:
        df2_sents.append(sent.text) 
        df2_labels.append(0)


In [17]:
len(df2_sents)

8743

In [18]:
df2_sents[:100]

['Britains little prince gets a name 25th July 2013 08:51 AM',
 'The little prince was in need of a name, and now, by George, hes got one.',
 'The announcement Wednesday that Prince William and his wife, Kate, had selected a moniker steeped in British history came as royal officials said the new parents were seeking quiet family time away from the flashbulbs and frenzy that accompanied the birth of their first child.',
 'While the news put to rest intense speculation over what name the couple would choose, the extreme interest around it illustrated how the 2dayold future heir is already on his way to a lifetime of fanfare and public glare.',
 'Kensington Palace said William and Kate were delighted to announce their sons name, adding that the baby will be known as His Royal Highness Prince George of Cambridge.',
 'The name George — borne by six kings — befits the boy now third in line to the British throne and was a favorite among British bookmakers, evoking the steadfastness of the que

In [19]:
sents = df1_sents.tolist() + df2_sents
y = df1_y.tolist() + df2_labels

In [18]:
len(sents)

16539

In [19]:
import pickle

In [20]:
data_path = 'Data/all_data2.pickle'

In [21]:
Data_ = {}

In [22]:
Data_['X'] = np.array(sents)
Data_['y'] = np.array(y)
Data_['stopwords'] = np.array(number_stopwords)

In [23]:
with open(data_path, 'wb') as file_:
    pickle.dump(Data_, file_, protocol=pickle.HIGHEST_PROTOCOL)

## Read pickle

In [24]:
data_path = 'Data/all_data2.pickle'

In [25]:
with open(data_path, 'rb') as file_:
    data_ = pickle.load(file_)

In [26]:
X = data_['X']
y = data_['y']
stopwords = data_['stopwords']

In [27]:
type(y)

numpy.ndarray

## Partition the data

In [28]:
 X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [29]:
len(X_), len(X_test)

(13231, 3308)

In [30]:
 X_tra, X_val, y_tra, y_val = train_test_split(X_, y_, test_size=0.20, random_state=42, stratify=y_)

In [31]:
type(X_tra), len(X_val)

(numpy.ndarray, 2647)

In [32]:
data_path = 'Data/prepared_data.pickle'

In [33]:
Data_ = {}

In [34]:
Data_['X_tra'] = X_tra
Data_['y_tra'] = y_tra
Data_['X_val'] = X_val
Data_['y_val'] = y_val
Data_['X_test'] = X_test
Data_['y_test'] = y_test
Data_['stopwords'] = np.array(number_stopwords)

In [35]:
with open(data_path, 'wb') as file_:
    pickle.dump(Data_, file_, protocol=pickle.HIGHEST_PROTOCOL)

# New way

In [129]:
df = pd.read_excel('Data/20181001-newindianexpress_sentence_classification_adjudicated_20181218.xlsx')

df = df[np.logical_not(np.isnan(np.array(df['label'])))]

len(df)

8337

In [130]:
df

,Unnamed: 0,url,sent_num,sentence,text,label
0,0,http://www.newindianexpress.com/states/andhra-...,1-1,Anantapur Lawyers to Revive Samaikya Agitation...,Anantapur Lawyers to Revive Samaikya Agitation...,2.0
1,1,http://www.newindianexpress.com/states/andhra-...,1-2,A meeting of lawyers was held in the town on S...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
2,2,http://www.newindianexpress.com/states/andhra-...,1-3,"Speaking to reporters later, advocates’ leader...",Anantapur Lawyers to Revive Samaikya Agitation...,0.0
3,3,http://www.newindianexpress.com/states/andhra-...,1-4,“Bifurcation is being done without taking into...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
4,4,http://www.newindianexpress.com/states/andhra-...,1-5,The Bill did not contain any proposals for dev...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
5,5,http://www.newindianexpress.com/states/andhra-...,1-6,"As part of their protests, the lawyers have de...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
6,6,http://www.newindianexpress.com/states/andhra-...,1-7,"“We appeal to all Samaikyandhra supporters, st...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
7,7,http://www.newindianexpress.com/states/andhra-...,1-8,"On January 28, the lawyers decided to take out...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
8,8,http://www.newindianexpress.com/states/andhra-...,1-9,State bar Association vice-president A Rami Re...,Anantapur Lawyers to Revive Samaikya Agitation...,2.0
9,9,http://www.newindianexpress.com/states/andhra-...,1-10,The lawyers body will also organise a meeting ...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0


In [131]:
for i, row in df.iterrows():
    row['sentence'] = re.sub("http\S*\s", "", row['sentence'])
    row['sentence'] = re.sub("((url)*\s*:\s*)*http\S*", "", row['sentence'])
    df['sentence'][i] = row['sentence']
    if row['sentence'] == "":
        df=df.drop([i])


/home/aorus/workspaces/simge/Master_Thesis/.env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [134]:
df.reset_index(drop=True)

,Unnamed: 0,url,sent_num,sentence,text,label
0,0,http://www.newindianexpress.com/states/andhra-...,1-1,Anantapur Lawyers to Revive Samaikya Agitation...,Anantapur Lawyers to Revive Samaikya Agitation...,2.0
1,1,http://www.newindianexpress.com/states/andhra-...,1-2,A meeting of lawyers was held in the town on S...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
2,2,http://www.newindianexpress.com/states/andhra-...,1-3,"Speaking to reporters later, advocates’ leader...",Anantapur Lawyers to Revive Samaikya Agitation...,0.0
3,3,http://www.newindianexpress.com/states/andhra-...,1-4,“Bifurcation is being done without taking into...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
4,4,http://www.newindianexpress.com/states/andhra-...,1-5,The Bill did not contain any proposals for dev...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
5,5,http://www.newindianexpress.com/states/andhra-...,1-6,"As part of their protests, the lawyers have de...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
6,6,http://www.newindianexpress.com/states/andhra-...,1-7,"“We appeal to all Samaikyandhra supporters, st...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
7,7,http://www.newindianexpress.com/states/andhra-...,1-8,"On January 28, the lawyers decided to take out...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
8,8,http://www.newindianexpress.com/states/andhra-...,1-9,State bar Association vice-president A Rami Re...,Anantapur Lawyers to Revive Samaikya Agitation...,2.0
9,9,http://www.newindianexpress.com/states/andhra-...,1-10,The lawyers body will also organise a meeting ...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0


In [135]:
df.to_csv(r'Data/preprocessed.csv')

In [133]:
df

,Unnamed: 0,url,sent_num,sentence,text,label
0,0,http://www.newindianexpress.com/states/andhra-...,1-1,Anantapur Lawyers to Revive Samaikya Agitation...,Anantapur Lawyers to Revive Samaikya Agitation...,2.0
1,1,http://www.newindianexpress.com/states/andhra-...,1-2,A meeting of lawyers was held in the town on S...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
2,2,http://www.newindianexpress.com/states/andhra-...,1-3,"Speaking to reporters later, advocates’ leader...",Anantapur Lawyers to Revive Samaikya Agitation...,0.0
3,3,http://www.newindianexpress.com/states/andhra-...,1-4,“Bifurcation is being done without taking into...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
4,4,http://www.newindianexpress.com/states/andhra-...,1-5,The Bill did not contain any proposals for dev...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
5,5,http://www.newindianexpress.com/states/andhra-...,1-6,"As part of their protests, the lawyers have de...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
6,6,http://www.newindianexpress.com/states/andhra-...,1-7,"“We appeal to all Samaikyandhra supporters, st...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
7,7,http://www.newindianexpress.com/states/andhra-...,1-8,"On January 28, the lawyers decided to take out...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
8,8,http://www.newindianexpress.com/states/andhra-...,1-9,State bar Association vice-president A Rami Re...,Anantapur Lawyers to Revive Samaikya Agitation...,2.0
9,9,http://www.newindianexpress.com/states/andhra-...,1-10,The lawyers body will also organise a meeting ...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0


In [50]:
doc_ids = set()
doc_sents_last = {}

In [53]:
for i, row in df.iterrows():
    doc_num, sent_num = row['sent_num'].split('-')
    doc_ids.add(doc_num)
    doc_sents_last[doc_num] = sent_num

In [87]:
import json_lines

In [88]:
v_data = []
with json_lines.open('Data/alldata.jsonl') as reader:
    for obj in reader:
        v_data.append(obj)
        

In [94]:
for item in v_data:
    if 'Crowd tries to attack US consulate' in item['sentence']:
        print(item)

{'id': '8570001', 'sentence': 'Crowd tries to attack US consulate in Pakistan 11th January 2009 11:56 PM Thousands of supporters of Pakistanis participate in a rally to condemn Israeli military attacks in Karachi.', 'label': 1, 'last': True}


7796

In [98]:
df[5900:6000]

,Unnamed: 0,url,sent_num,sentence,text,label
12471,12471,http://www.newindianexpress.com/opinions/2010/...,849-48,“You are afraid we shall be cut off in Chamdo?...,The two sides of Ngapoi 14th January 2010 11:4...,0.0
12472,12472,http://www.newindianexpress.com/opinions/2010/...,849-49,"This was done on May 23, 1950.",The two sides of Ngapoi 14th January 2010 11:4...,0.0
12473,12473,http://www.newindianexpress.com/opinions/2010/...,849-50,The Dalai Lama was not even informed.,The two sides of Ngapoi 14th January 2010 11:4...,0.0
12474,12474,http://www.newindianexpress.com/opinions/2010/...,849-51,The Tibetan seals were forged by the Chinese a...,The two sides of Ngapoi 14th January 2010 11:4...,0.0
12475,12475,http://www.newindianexpress.com/opinions/2010/...,849-52,Ngapo did not even protest.,The two sides of Ngapoi 14th January 2010 11:4...,0.0
12476,12476,http://www.newindianexpress.com/opinions/2010/...,849-53,The Dalai Lama eventually denounced the agreem...,The two sides of Ngapoi 14th January 2010 11:4...,0.0
12477,12477,http://www.newindianexpress.com/opinions/2010/...,849-54,While the present government in Beijing stated...,The two sides of Ngapoi 14th January 2010 11:4...,0.0
12478,12478,http://www.newindianexpress.com/opinions/2010/...,849-55,"Unfortunately, this policy of appeasement may ...",The two sides of Ngapoi 14th January 2010 11:4...,0.0
12479,12479,http://www.newindianexpress.com/opinions/2010/...,849-56,About the author: Claude Arpi is a French-born...,The two sides of Ngapoi 14th January 2010 11:4...,0.0
12480,12480,http://www.newindianexpress.com/states/odisha/...,850-1,Race for ministerial berths in BJD begins 19th...,Race for ministerial berths in BJD begins 19th...,0.0


In [70]:
jslines=[]
last_=False
for i, row in df.iterrows():
    doc_num, sent_num = row['sent_num'].split('-')
    row['sentence'] = re.sub("http\S*\s", "", row['sentence'])
    sent = re.sub("((url)*\s*:\s*)*http\S*", "", row['sentence'])
    if not sent == "":
        if sent_num == doc_sents_last[doc_num]:
            last_=True
#             print(doc_num, sent_num)
        jslines.append({'id': str(doc_num)+'000'+str(sent_num),
                        'sentence': sent,
                        'label': int(row['label']),
                        'last':last_})


In [ ]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

len(X_), len(X_test)

In [71]:
jslines

[{'id': '10001',
  'sentence': 'Anantapur Lawyers to Revive Samaikya Agitation, to Lay Siege to Courts Today 27th January 2014 09:58 AM Renewing the Samikyandhra agitation, the lawyers of the district have decided to take on a series of protests starting with laying siege to courts on Monday.',
  'label': 2,
  'last': False},
 {'id': '10002',
  'sentence': 'A meeting of lawyers was held in the town on Sunday in which the members of the Anantapur Bar Association in which various protests were planned.',
  'label': 0,
  'last': False},
 {'id': '10003',
  'sentence': 'Speaking to reporters later, advocates’ leader Valmiki Gangadhar reiterated their demand that the state should be kept united.',
  'label': 0,
  'last': False},
 {'id': '10004',
  'sentence': '“Bifurcation is being done without taking into consideration the opinion of the people of Seemandhra region.',
  'label': 0,
  'last': False},
 {'id': '10005',
  'sentence': 'The Bill did not contain any proposals for developing the ba

In [72]:
import json

with open('Data/alldata.jsonl', 'w') as outfile:
    for entry in jslines:
        json.dump(entry, outfile)
        outfile.write('\n')
        